
# Category-level TF‑IDF + Themes (Best‑of‑Both, Canonical Pipeline)

This notebook merges:
- the **original exploratory TF‑IDF outputs** (tables you relied on),
- with the **correct, leak‑free canonical token pipeline**.

Key invariants:
- `df_tokens` is the *only* dataframe used for TF‑IDF, NMF, contrast.
- Hard stopwords cannot leak (asserted).
- All original TF‑IDF displays are preserved.


In [1]:

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)


## 1. Load data (replace with real loads)

In [2]:
tokens1 = pd.read_csv('/Users/matt.fritz/Desktop/Research Insights/Essay Tokens jan302024topresent_1.csv')  # project_id, tokens
tokens2 = pd.read_csv('/Users/matt.fritz/Desktop/Research Insights/Essay Tokens jan302024topresent_3.csv')  # project_id, tokens
tokens3 = pd.read_csv('/Users/matt.fritz/Desktop/Research Insights/Essay Tokens jan302024topresent_5.csv')  # project_id, tokens
tokens4 = pd.read_csv('/Users/matt.fritz/Desktop/Research Insights/Essay Tokens jan302024topresent_7.csv')  # project_id, tokens
tokens5 = pd.read_csv('/Users/matt.fritz/Desktop/Research Insights/Essay Tokens jan302024topresent_9.csv')  # project_id, tokens
tokens  = pd.concat(
    [tokens1, tokens2, tokens3, tokens4, tokens5],
    ignore_index=True
)

projects = pd.read_csv('/Users/matt.fritz/Desktop/Research Insights/ProjSuccess_ProjectRawData_260116_clean.csv')  # project metadata

projects['posted_date'] = pd.to_datetime(projects['posted_date'])

# Keep only projects with tokens
df = projects.merge(tokens, on='project_id', how='inner')
df.shape

CATEGORY_FIELD = "grade_band"

## 2. Explode tokens (raw)

In [3]:

df["token"] = df["tokens"].str.split(",")
df = df.explode("token", ignore_index=True)
df["token"] = df["token"].astype(str).str.lower().str.strip()

df = df.drop_duplicates(subset=["project_id", "token"])
df = df[df["token"].ne("")]

df.head()


,project_id,funded_date,materials_vendors_count,material_cost,has_eligible_match,fy25_historical_efs_status,expiration_date,is_favorite_or_exciting,is_professional_development,is_student_led,is_teachers_first_posted_project,ariba_line_items_count,teachers_nth_posted_project,project_received_government_grant,percentage_free_lunch_at_time_of_posting,metro_type_at_time_of_posting,school_zip,school_id_at_time_of_posting,school_enrollment,school_is_historically_underrepresented_race,school_is_low_income,school_is_racially_predominant,school_is_underserved_rural,school_year_open,school_percent_black_imputed,school_percent_latinx_imputed,school_percent_asian_imputed,school_percent_white_imputed,sf_district_account_date_discouraging_dc_date_1,teacher_created_date,teacher_days_to_activation,teacher_first_project_posted_date,teacher_photo_published,teacher_has_profile_photo,teacher_is_teacher_of_color,teacher_gender,teacher_start_teaching_year,posted_date,grade_band,project_category,total_cost,teacher_id,tokens,token
0,8173551,2024-01-31,6,346.1,No,Yes,2024-05-24,No,No,No,Yes,14.0,1,No,95.0,Urban,39202,12275.0,1279.0,Yes,Yes,Yes,No,Early (prior to 8/16),94,2,0,1,NaN,2022-08-01,547.0,2024-01-31,Yes,No,Yes,NaN,NaN,2024-01-31,Grades 9-12,Art Supplies,493.92,8560900,"cards,enjoyed,holidays,now,back,finish,strong,...",cards
1,8173551,2024-01-31,6,346.1,No,Yes,2024-05-24,No,No,No,Yes,14.0,1,No,95.0,Urban,39202,12275.0,1279.0,Yes,Yes,Yes,No,Early (prior to 8/16),94,2,0,1,NaN,2022-08-01,547.0,2024-01-31,Yes,No,Yes,NaN,NaN,2024-01-31,Grades 9-12,Art Supplies,493.92,8560900,"cards,enjoyed,holidays,now,back,finish,strong,...",enjoyed
2,8173551,2024-01-31,6,346.1,No,Yes,2024-05-24,No,No,No,Yes,14.0,1,No,95.0,Urban,39202,12275.0,1279.0,Yes,Yes,Yes,No,Early (prior to 8/16),94,2,0,1,NaN,2022-08-01,547.0,2024-01-31,Yes,No,Yes,NaN,NaN,2024-01-31,Grades 9-12,Art Supplies,493.92,8560900,"cards,enjoyed,holidays,now,back,finish,strong,...",holidays
3,8173551,2024-01-31,6,346.1,No,Yes,2024-05-24,No,No,No,Yes,14.0,1,No,95.0,Urban,39202,12275.0,1279.0,Yes,Yes,Yes,No,Early (prior to 8/16),94,2,0,1,NaN,2022-08-01,547.0,2024-01-31,Yes,No,Yes,NaN,NaN,2024-01-31,Grades 9-12,Art Supplies,493.92,8560900,"cards,enjoyed,holidays,now,back,finish,strong,...",now
4,8173551,2024-01-31,6,346.1,No,Yes,2024-05-24,No,No,No,Yes,14.0,1,No,95.0,Urban,39202,12275.0,1279.0,Yes,Yes,Yes,No,Early (prior to 8/16),94,2,0,1,NaN,2022-08-01,547.0,2024-01-31,Yes,No,Yes,NaN,NaN,2024-01-31,Grades 9-12,Art Supplies,493.92,8560900,"cards,enjoyed,holidays,now,back,finish,strong,...",back


## 3. Canonical token dataframe + hard stopwords

In [4]:

df_tokens = df.copy()

HARD_STOPWORDS = {
    "kindergarten","preschoolers","freshmen","freshman","sophomores","olds",
    "preschool","firsties","kinders","fourth","preschooler","kindergartners",
    "winn","dixie","juniors","kindergarteners","seniors","prekindergarten",
    "schoolers","senior","kindergartner","learns",
}
HARD_STOPWORDS = {w.lower() for w in HARD_STOPWORDS}

df_tokens = df_tokens[~df_tokens["token"].isin(HARD_STOPWORDS)]

assert not df_tokens["token"].isin(HARD_STOPWORDS).any(), "Hard stopwords leaked"

print("✅ Hard stopwords removed")
print("Rows:", len(df_tokens), "Unique tokens:", df_tokens["token"].nunique())


✅ Hard stopwords removed
Rows: 5169856 Unique tokens: 40209


## 4. Corpus‑level token pruning

In [5]:

MAX_PROJECT_COVERAGE = 0.50
MAX_CATEGORY_COVERAGE = 1.00
MIN_PROJECT_COUNT = 3

n_projects = df_tokens["project_id"].nunique()
n_categories = df_tokens[CATEGORY_FIELD].nunique()

token_stats = (
    df_tokens
    .groupby("token")
    .agg(
        project_count=("project_id", "nunique"),
        category_count=(CATEGORY_FIELD, "nunique"),
    )
)

token_stats["project_coverage"] = token_stats["project_count"] / n_projects
token_stats["category_coverage"] = token_stats["category_count"] / n_categories

bad_tokens = token_stats.index[
    (token_stats["project_coverage"] > MAX_PROJECT_COVERAGE)
    | (token_stats["category_coverage"] > MAX_CATEGORY_COVERAGE)
    | (token_stats["project_count"] < MIN_PROJECT_COUNT)
]

df_tokens = df_tokens[~df_tokens["token"].isin(bad_tokens)]

print(f"Filtered {len(bad_tokens):,} corpus‑level tokens")
print("Remaining unique tokens:", df_tokens["token"].nunique())


Filtered 19,899 corpus‑level tokens
Remaining unique tokens: 20310


In [6]:
token_stats[["project_coverage", "category_coverage", "project_count"]].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])


,project_coverage,category_coverage,project_count
count,40209.000000,40209.000000,40209.000000
mean,0.001410,0.570712,128.574598
std,0.009316,0.326152,849.544247
min,0.000011,0.250000,1.000000
50%,0.000033,0.500000,3.000000
75%,0.000175,1.000000,16.000000
90%,0.001316,1.000000,120.000000
95%,0.004514,1.000000,411.600000
99%,0.030451,1.000000,2776.920000
max,0.358014,1.000000,32648.000000


## 5. Category‑level TF‑IDF (core table)

In [7]:

cat_token = (
    df_tokens
    .groupby([CATEGORY_FIELD, "token"])
    .agg(k_projects=("project_id", "nunique"))
    .reset_index()
)

n_cat_projects = (
    df_tokens
    .groupby(CATEGORY_FIELD)["project_id"]
    .nunique()
    .rename("n_projects")
    .reset_index()
)

cat_token = cat_token.merge(n_cat_projects, on=CATEGORY_FIELD, how="left")

cat_token["tf"] = cat_token["k_projects"] / cat_token["n_projects"]

n_categories = cat_token[CATEGORY_FIELD].nunique()

token_cat_counts = (
    cat_token
    .groupby("token")[CATEGORY_FIELD]
    .nunique()
    .rename("cat_count")
)

cat_token = cat_token.merge(token_cat_counts, on="token", how="left")
cat_token["idf"] = np.log(n_categories / (1 + cat_token["cat_count"]))
cat_token["tfidf"] = cat_token["tf"] * cat_token["idf"]

cat_token.head()


,grade_band,token,k_projects,n_projects,tf,cat_count,idf,tfidf
0,Grades 3-5,aaa,5,26018,0.000192,4,-0.223144,-0.000043
1,Grades 3-5,aac,17,26018,0.000653,4,-0.223144,-0.000146
2,Grades 3-5,aapi,1,26018,0.000038,4,-0.223144,-0.000009
3,Grades 3-5,aaron,4,26018,0.000154,4,-0.223144,-0.000034
4,Grades 3-5,aba,9,26018,0.000346,4,-0.223144,-0.000077


## 6. TF‑IDF exploratory outputs (restored)

In [8]:

# Global top TF‑IDF
display(cat_token.sort_values("tfidf", ascending=False).head(50))


,grade_band,token,k_projects,n_projects,tf,cat_count,idf,tfidf
51743,Grades 9-12,welding,35,16380,0.002137,1,0.693147,0.001481
36063,Grades 9-12,biotechnology,29,16380,0.001770,1,0.693147,0.001227
48492,Grades 9-12,shakespeare,52,16380,0.003175,2,0.287682,0.000913
41091,Grades 9-12,gatsby,21,16380,0.001282,1,0.693147,0.000889
38891,Grades 9-12,diploma,43,16380,0.002625,2,0.287682,0.000755
67550,Grades PreK-2,tonies,84,32806,0.002561,2,0.287682,0.000737
39056,Grades 9-12,dissolved,17,16380,0.001038,1,0.693147,0.000719
42206,Grades 9-12,immigrate,39,16380,0.002381,2,0.287682,0.000685
40767,Grades 9-12,forensics,37,16380,0.002259,2,0.287682,0.000650
47247,Grades 9-12,regents,37,16380,0.002259,2,0.287682,0.000650


In [9]:

# Top TF‑IDF tokens per category (original core output)
TOP_N = 15

display(
    cat_token
    .sort_values([CATEGORY_FIELD, "tfidf"], ascending=[True, False])
    .groupby(CATEGORY_FIELD)
    .head(TOP_N)[
        [CATEGORY_FIELD, "token", "tf", "idf", "tfidf", "k_projects", "n_projects"]
    ]
)


,grade_band,token,tf,idf,tfidf,k_projects,n_projects
14992,Grades 3-5,storyworks,0.001653,0.287682,0.000475,43,26018
9415,Grades 3-5,matilda,0.000500,0.693147,0.000346,13,26018
845,Grades 3-5,arrays,0.000884,0.287682,0.000254,23,26018
4073,Grades 3-5,departmentalized,0.000884,0.287682,0.000254,23,26018
8263,Grades 3-5,ivan,0.000807,0.287682,0.000232,21,26018
9751,Grades 3-5,miraculous,0.000307,0.693147,0.000213,8,26018
567,Grades 3-5,amira,0.000653,0.287682,0.000188,17,26018
3806,Grades 3-5,dahl,0.000653,0.287682,0.000188,17,26018
3527,Grades 3-5,cozying,0.000269,0.693147,0.000186,7,26018
16324,Grades 3-5,tulane,0.000269,0.693147,0.000186,7,26018


## 7. NMF themes per category

In [10]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

N_THEMES = 5
TOP_WORDS_PER_THEME = 10
MIN_PROJECTS_PER_CATEGORY = 100

themes = []

for cat, sub in df_tokens.groupby(CATEGORY_FIELD):
    docs = sub.groupby("project_id")["token"].apply(lambda x: " ".join(x))
    if len(docs) < MIN_PROJECTS_PER_CATEGORY:
        continue

    vec = TfidfVectorizer(min_df=5, max_df=0.9)
    X = vec.fit_transform(docs)

    nmf = NMF(n_components=N_THEMES, random_state=42, init="nndsvd")
    H = nmf.fit(X).components_
    vocab = np.array(vec.get_feature_names_out())

    for i, w in enumerate(H):
        themes.append({
            "category": cat,
            "theme_id": i + 1,
            "tokens": vocab[np.argsort(w)[::-1][:TOP_WORDS_PER_THEME]].tolist()
        })

themes_df = pd.DataFrame(themes)
themes_df.head()


,category,theme_id,tokens
0,Grades 3-5,1,"[all, able, work, class, some, not, keep, time..."
1,Grades 3-5,2,"[hands, skills, problem, explore, creativity, ..."
2,Grades 3-5,3,"[environment, thrive, create, feel, focus, emo..."
3,Grades 3-5,4,"[books, reading, read, library, book, love, re..."
4,Grades 3-5,5,"[seating, flexible, comfortable, options, sit,..."


## 8. Contrast + prevalence scoring

In [11]:

cat_token_projects = (
    df_tokens
    .groupby([CATEGORY_FIELD, "token"])
    .agg(token_project_count=("project_id", "nunique"))
    .reset_index()
)

cat_projects = (
    df_tokens
    .groupby(CATEGORY_FIELD)["project_id"]
    .nunique()
    .rename("category_project_count")
    .reset_index()
)

cat_token_projects = cat_token_projects.merge(cat_projects, on=CATEGORY_FIELD, how="left")
cat_token_projects["tf"] = (
    cat_token_projects["token_project_count"]
    / cat_token_projects["category_project_count"]
)

cat_token_projects = cat_token_projects.set_index([CATEGORY_FIELD, "token"])

def score_theme(tokens, category):
    try:
        tf_in = cat_token_projects.loc[(category, tokens), "tf"].mean()
        tf_out = (
            cat_token_projects
            .loc[pd.IndexSlice[:, tokens], "tf"]
            .reset_index()
            .query(f"{CATEGORY_FIELD} != @category")
            .groupby("token")["tf"]
            .mean()
            .mean()
        )
        count = cat_token_projects.loc[(category, tokens), "token_project_count"].sum()
        total = cat_projects.set_index(CATEGORY_FIELD).loc[category, "category_project_count"]
        return pd.Series({
            "contrast": tf_in - tf_out,
            "theme_project_count": int(count),
            "theme_project_pct": count / total,
        })
    except KeyError:
        return pd.Series({
            "contrast": 0.0,
            "theme_project_count": 0,
            "theme_project_pct": 0.0,
        })

themes_df[["contrast","theme_project_count","theme_project_pct"]] = (
    themes_df.apply(lambda r: score_theme(r["tokens"], r["category"]), axis=1)
)


## 9. Theme outputs

In [12]:

display(
    themes_df
    .sort_values("contrast", ascending=False)
    .head(20)[
        ["category","theme_id","contrast","theme_project_pct","tokens"]
    ]
)


,category,theme_id,contrast,theme_project_pct,tokens
16,Grades PreK-2,2,0.042793,1.574560,"[skills, hands, play, motor, explore, problem,..."
14,Grades 9-12,5,0.039523,1.480098,"[equipment, program, community, team, skills, ..."
18,Grades PreK-2,4,0.038419,1.541639,"[reading, books, read, library, readers, book,..."
3,Grades 3-5,4,0.033518,1.470290,"[books, reading, read, library, book, love, re..."
12,Grades 9-12,3,0.020435,1.521673,"[environment, focus, space, feel, emotional, c..."
4,Grades 3-5,5,0.019400,0.730417,"[seating, flexible, comfortable, options, sit,..."
17,Grades PreK-2,3,0.016718,0.938456,"[space, comfortable, seating, safe, feel, flex..."
10,Grades 9-12,1,0.009558,2.134554,"[class, all, some, able, keep, work, not, want..."
6,Grades 6-8,2,0.007763,1.109144,"[hands, explore, problem, world, solving, crea..."
5,Grades 6-8,1,0.003882,2.170003,"[all, class, able, some, not, work, out, want,..."
